## PDF Q&A RAG from Scratch with Google Gemini and ChromaDB ##

Steps that we will follow. I will not use Langchain or any ready-made library for this video to help you understand the core basics of the workflow.

1. Using requests, download the pdf of interest
2. load the pdf
3. chunk the pdf. we will create our own splitter.
4. use Google Gemini embedding function for creating embeddings
5. Create a chroma db collection with a name and embedding function.
6. ingest documents into chroma.
7. basis the query, find relevant passage by making a call to chroma.
8. send the passages as context and question by the user to gemini pro 1.5 latest.
9. output results.

In [ ]:
!pip install requests
!pip install PyPDF2
!pip install google-generativeai
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 11.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7

In [ ]:
# configure google gemini
import google.generativeai as genai

from google.colab import userdata
genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

In [ ]:
for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
import requests
import os

# create a method called download_pdf which will take a url and a save_path
# and download the pdf from the url and save it in the path specified.

def download_pdf(url, save_path):

  response = requests.get(url)
  response.raise_for_status()

  with open(save_path, "wb") as file:
    file.write(response.content)

In [ ]:
url = "https://arxiv.org/abs/1706.03762"
save_path = "attention_is_all_you_need.pdf"

download_pdf(url, save_path)

In [ ]:
from PyPDF2 import PdfReader

# write a function called load_pdf which will take a pdf file as file_path and
# extract the text from it using PdfReader.

def load_pdf(file_path):
  pdf_reader = PdfReader(file_path)
  text = ""
  for page in pdf_reader.pages:
    text += page.extract_text()
  return text


In [ ]:
pdf_text = load_pdf(save_path)


In [ ]:
pdf_text[:500]

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaise'

In [ ]:
len(pdf_text)

39472

In [ ]:
# build a function called split_text_recursively which will take
# text which is the original text which needs to be split.
# the max_length of the chunk and the chunk_overlap to specify how much overlap is allowed
# between two chunks.

def split_text_recursively(text, max_length=1000, chunk_overlap=0):
  chunks = []
  start = 0 # start at the beginning
  text_length = len(text) # figure out how long is the text provided.
  while start < text_length: # keep going until we have looked at all the text
    end = start + max_length
    if end < text_length : # if we are not yet at the end of the text
      end = text.rfind(' ', start, end) + 1 # end the chunk at an empty space.

      if end <= start: # if there is no space, then just split at the max length.
        end = start + max_length
    chunk = text[start:end].strip() # take the text from start to end and remove extra spaces.

    if chunk:
      chunks.append(chunk)

    start = end - chunk_overlap  # moving the start position forward minus any overlaps.

    if start >= text_length: # if we have reached the end of the text
      break
  return chunks





In [ ]:
chunks = split_text_recursively(pdf_text, max_length=2000, chunk_overlap=200)

In [ ]:
len(chunks)

23

In [ ]:
chunks[0]

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and con

In [ ]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=userdata.get("GOOGLE_API_KEY"))
client = chromadb.PersistentClient(path="embeddings/gemini")

collection = client.get_or_create_collection(name="pdf_rag", embedding_function=google_ef)


for i, d in enumerate(chunks):
  collection.add(documents=[d], ids=[str(i)])

In [ ]:
collection.count()

23

In [ ]:
def find_relevant_context(query, db, n_results=3):
  results = db.query(query_texts=[query], n_results=n_results)
  return results

In [ ]:
results = find_relevant_context("role of encoders", collection )